### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv("./project.env")

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Part 1: Access the New York Times API (35 points)

In [3]:
# Set the base URL
base_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
# filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
filter_query = 'section_name:("Movies") AND type_of_material:("Review") AND headline:("love")'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

- query_url is correctly constructed (2 points).

In [4]:
# Build URL
params = {
    "api-key": nyt_api_key,
    "fq": filter_query,
    "sort": sort,
    "fl": field_list,
    "begin_date": begin_date,
    "end_date": end_date
}
request_url = requests.Request('GET', base_url, params=params).prepare().url

- An empty list reviews_list is created (1 point).

In [5]:
# Create an empty list to store the reviews
reviews_list = []

- A for loop is created to loop through 20 times (3 points).
  - The query_url is extended to include a page (1 point).
  - A GET request is made to retrieve results and the JSON data is stored in a variable called reviews (4 points).
  - A 12-second interval is used between queries (2 points).
  - A try-except clause is used (2 points).
    - Inside the try clause, there is a loop to loop through the reviews["response"]["docs"] list (3 points).
    - The reviews results are correctly appended to reviews_list (2 points).
    - The query page number is printed (1 point).
    - The except clause prints out the page number that had no results, then breaks from the loop (2 points).

In [6]:
# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time\
    request_by_page_url = f"{request_url}&page={page}"

    # Make a "GET" request and retrieve the JSON
    request = requests.get(request_by_page_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        reviews = request.get("response", {}).get("docs", [])
        if not reviews:
            fault = request.get("fault", {}).get("faultstring", "")
            raise Exception(f"No reviews found. {fault}")

        reviews_list.extend(reviews)

        # Print the page that was just retrieved
        print(f"Retrieved page {page}, {len(reviews)} reviews saved")

    # Print the page number that had no results then break from the loop
    except Exception as error:
        print(f"Page {page}: {error}")
        break


Retrieved page 0, 10 reviews saved
Retrieved page 1, 10 reviews saved
Retrieved page 2, 10 reviews saved
Retrieved page 3, 10 reviews saved
Retrieved page 4, 10 reviews saved
Retrieved page 5, 10 reviews saved
Retrieved page 6, 10 reviews saved
Retrieved page 7, 10 reviews saved
Retrieved page 8, 10 reviews saved
Retrieved page 9, 10 reviews saved
Retrieved page 10, 10 reviews saved
Retrieved page 11, 10 reviews saved
Retrieved page 12, 10 reviews saved
Retrieved page 13, 10 reviews saved
Retrieved page 14, 10 reviews saved
Retrieved page 15, 10 reviews saved
Retrieved page 16, 10 reviews saved
Retrieved page 17, 10 reviews saved
Retrieved page 18, 10 reviews saved
Retrieved page 19, 10 reviews saved


- json.dumps with the argument indent=4 is used to preview the first five results (2 points).

In [7]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

- reviews_list is converted to a Pandas DataFrame using json_normalize() (2 points).

In [8]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


- The title is extracted from the "headline.main" column and is saved in a new column "title" (3 points).

In [9]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df["title"] = reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

- The "keywords" column is correctly converted to string data using the supplied extract_keywords function (3 points).

In [10]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)

- A list called titles is created from the "title" column using to_list() (2 points).

In [11]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df["title"].to_list()

### Access The Movie Database API

In [12]:
# Prepare The Movie Database query
base_url = "https://api.themoviedb.org/3/search/movie?query="
movie_base_url = "https://api.themoviedb.org/3/movie/"
tmdb_key_string = "&api_key=" + tmdb_api_key

- An empty list called tmdb_movies_list is created (1 point).
- A variable called request_counter is created and assigned the value of 1 (1 point).
- A for loop is created to loop through the titles list (2 points).
  - request_counter is incremented by 1 (1 point).
  - time.sleep(1) when request_counter reaches a multiple of 50 (3 points).
  - A GET request that sends the title to The Movie Database search is performed, and the JSON results are retrieved (4 points).
  - A try-except clause is used (3 points).
    - The movie ID is collected from the first result and saved as a variable (2 points).
    - A GET request is made using the movie query URL and movie ID to retrieve the full movie details in JSON format (4 points).
    - The genre names are extracted from the results into a list called genres (2 points).
    - The spoken_languages' English names are extracted from the results into a list called spoken_languages (2 points).
    - The production_countries' names are extracted from the results into a list called production_countries (2 points).
    - A dictionary is created with the specified 15 fields (4 points).
    - The results dictionary is appended to the tmdb_movies_list list (3 points).
    - A message is printed with the name of the movie to indicate that the title was found. (1 point)
  - The except clause prints out a statement if a movie is not found (1 point).

In [27]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 0

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0: time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    request = requests.get(base_url + title + tmdb_key_string).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie is not found.
    try:
        # Get movie id
        movie_id = request["results"][0]["id"]

        # Make a request for a the full movie details
        movie_url = f"{movie_base_url}{movie_id}?{tmdb_key_string}"

        # Execute "GET" request with url
        movie_request = requests.get(movie_url).json()
        
        # Extract the genre names into a list
        movie_genres = [genre["name"] for genre in movie_request["genres"]]

        # Extract the spoken_languages' English name into a list
        movie_languages = [lang["english_name"] for lang in movie_request["spoken_languages"]]

        # Extract the production_countries' name into a list
        movie_countries = [country["name"] for country in movie_request["production_countries"]]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            "title": movie_request["title"],
            "original_title": movie_request["original_title"],
            "budget": movie_request["budget"],
            "original_language": movie_request["original_language"],
            "homepage": movie_request["homepage"],
            "overview": movie_request["overview"],
            "popularity": movie_request["popularity"],
            "runtime": movie_request["runtime"],
            "revenue": movie_request["revenue"],
            "release_date": movie_request["release_date"],
            "vote_average": movie_request["vote_average"],
            "vote_count": movie_request["vote_count"],
            "genres": movie_genres,
            "spoken_languages": movie_languages,
            "production_countries": movie_countries
        })

        # Print out the title that was found
        print(f"Found: {title}")
    except:
        print(f"NOT Found: {title}")



Found: The Attachment Diaries
NOT Found: What’s Love Got to Do With It?’ Probably a Lo
Found: You Can Live Forever
Found: A Tourist’s Guide to Love
Found: Other People’s Children
Found: One True Loves
Found: The Lost Weekend: A Love Story
Found: A Thousand and One
Found: Your Place or Mine
Found: Love in the Time of Fentanyl
Found: Pamela, a Love Story
Found: In From the Side
Found: After Love
Found: Alcarràs
Found: Nelly & Nadine
Found: Lady Chatterley’s Lover
Found: The Sound of Christmas
Found: The Inspection
Found: Bones and All
Found: My Policeman
Found: About Fate
Found: Waiting for Bojangles
Found: I Love My Dad
Found: A Love Song
Found: Alone Together
Found: Art of Love
Found: The Wheel
Found: Thor: Love and Thunder
Found: Both Sides of the Blade
Found: Fire of Love
Found: Love & Gelato
Found: Stay Prayed Up
Found: Benediction
Found: Dinner in America
Found: In a New York Minute
Found: Anaïs in Love
Found: I Love America
Found: See You Then
Found: La Mami
Found: Love After Love

- The first five results are previewed using json.dumps with the argument indent=4 (2 points).

In [28]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))


[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 0.765,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

- The results are converted to a DataFrame called tmdb_df with pd.DataFrame() (2 points).

In [29]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",0.765,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",23.028,96,15055,2023-03-24,6.600,40,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",17.027,96,0,2023-04-21,6.303,170,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",7.627,104,84178,2022-09-21,6.812,194,"[Drama, Comedy]","[French, English]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,12.419,100,37820,2023-04-07,6.451,81,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",12.209,130,0,2017-01-11,5.500,194,[Drama],"[English, Portuguese]",[United States of America]
150,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,7.768,83,0,2017-01-14,6.000,115,[Comedy],[French],"[Belgium, France]"
151,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,2.912,103,0,2016-12-02,6.300,24,"[Drama, Romance]",[English],[Canada]
152,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",11.641,111,413844,2017-03-28,6.163,248,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]"


### Merge and Clean the Data for Export

- The New York Times reviews and TMDB DataFrames are merged on the title column (4 points).

In [30]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_movies_df, reviews_df, on="title", how="left")
merged_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",0.765,102,0,2021-10-07,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",23.028,96,15055,2023-03-24,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",17.027,96,0,2023-04-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",7.627,104,84178,2022-09-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,12.419,100,37820,2023-04-07,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",12.209,130,0,2017-01-11,...,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
150,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,7.768,83,0,2017-01-14,...,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
151,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,2.912,103,0,2016-12-02,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
152,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",11.641,111,413844,2017-03-28,...,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None


- A list called columns_to_fix is created to store the names of the genres, spoken_languages, and production_countries columns (2 points).
- A list is created called characters_to_remove containing [, ], and ' (2 points).
- A for loop is created to loop through columns_to_fix (2 points).
  - The columns to fix are converted to the string data type (2 points).
  - characters_to_remove is looped through to remove the characters from the string using the Pandas str.replace() method (4 points).
- The head of the updated DataFrame is displayed to confirm the list characters were removed (2 points).

In [31]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genres", "spoken_languages", "production_countries"]

# Create a list of characters to remove
chars_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:

    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for char in chars_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, "")

# Display the fixed DataFrame
merged_df.head()


,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",0.765,102,0,2021-10-07,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",23.028,96,15055,2023-03-24,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",17.027,96,0,2023-04-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",7.627,104,84178,2022-09-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,12.419,100,37820,2023-04-07,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None


- The byline.person column is dropped (2 points).

In [32]:
# Drop "byline.person" column
merged_df.drop(columns="byline.person", inplace=True)

- Duplicate rows are deleted (1 point).
- The DataFrame index is reset (1 point).

In [33]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)

- The DataFrame is exported to a CSV file without the index (3 points).

In [34]:
# Export data to CSV without the index
merged_df.to_csv("output/retrieved_movie_data.csv", index=False)